In [5]:
from pymnet import *
from IPython.display import display, HTML
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import pandas as pd
import mglearn
import math
import random
import networkx as nx
from sympy import *      
from mpl_toolkits.mplot3d import Axes3D
import numpy as np        
from matplotlib import cm
import pylab
import time

In [34]:
def A_layer_config(state, node, edge, Max, Min) :   # A_layer 구성요소 A_layer_config(state = [1,2], node = 2048, edge = 5, Max = 2, Min = -2)
    global A_state, A_node, A_edge, MAX, MIN
    A_state = state                                 # state = [   ]  kinds of states
    A_node = node
    A_edge = edge
    MAX = Max
    MIN = Min
    return A_state, A_node, A_edge, MAX, MIN
    
def B_layer_config(state, node, edge, inter_edge) : # B_layer 구성요소 B_layer_config(state = [-1], node = 2048, edge = 5, inter_edge= 1)
    global B_state, B_node, B_edge, inter_edges
    B_state = state                                 # state = [   ]  kinds of states
    B_node = node
    B_edge = edge
    inter_edges = inter_edge  
    return B_state, B_node, B_edge, inter_edges

def making_layer_A_random_regular() :               # A_layer random_regular network
    global A, A_edges
    A = np.array(A_state * int(A_node/len(A_state)), int)
    random.shuffle(A)
    A_edges = nx.random_regular_graph(A_edge, A_node, seed=None)
    return A, A_edges, MAX, MIN

def making_layer_B_random_regular() :               # B_layer random_regular network
    global B, B_edges
    B = np.array(B_state * int(B_node/len(B_state)), int)
    random.shuffle(B)
    B_edges = nx.random_regular_graph(B_edge, B_node, seed=None)
    return B, B_edges

def making_layer_A_barabasi_albert() :     # A_layer 바바라시-알버트 네트워크
    global A, A_edges
    A = np.array(A_state * int(A_node/len(A_state)), int)
    random.shuffle(A)
    A_edges = nx.barabasi_albert_graph(A_node, A_edge, seed=None)
    return A, A_edges, MAX, MIN

def making_layer_B_barabasi_albert() :     # B_layer 바바라시-알버트 네트워크
    global B, B_edges
    B = np.array(B_state * int(B_node/len(B_state)), int)
    random.shuffle(B)
    B_edges = nx.barabasi_albert_graph(B_node, B_edge, seed=None)
    return B, B_edges

def making_interconnected_edges() :        # 
    global AB_edges, AB_neighbor
    AB_edges = []
    AB_neighbor = []
    for i in range(int(A_node/inter_edges)) :
        for j in range(inter_edges) :
            connected_A_node = np.array(A_edges.nodes).reshape(-1, inter_edges)[i][j]
            AB_neighbor.append(connected_A_node)
            AB_edges.append((i, connected_A_node))
    AB_neighbor = np.array(AB_neighbor).reshape(-1, inter_edges)
    return AB_edges, AB_neighbor

def select_layer_A_model(a) :
    if a == 1 :
        making_layer_A_random_regular()
    elif a == 2 :
        making_layer_A_barabasi_albert()
    return A, A_edges, MAX, MIN
        
    
def select_layer_B_model(b) :
    if b == 1 :
        making_layer_B_random_regular()
    elif b == 2 :
        making_layer_B_barabasi_albert()
    return B, B_edges

def A_layer_node_left(a, MIN) :
    if a >= MIN : 
        if a == MIN :
            a = a
        elif a < 0 or a > 1 :
            a = a - 1
        elif a == 1 :
            a = -1
    elif a < MIN : 
        a = MIN
    return a
            
def A_layer_node_right(a, MAX) :
    if a <= MAX : 
        if a == MAX :
            a = a
        elif a > 0 or a < -1 :
            a = a + 1
        elif a == -1 :
            a = 1
    elif a > MAX : 
        a = MAX
    return a

def A_layer_persuasion_function(a, b) :  # A layer 중에서 same orientation 에서 일어나는  변동 현상
    z = random.random()
    if z < prob_p :
        if a > 0 :
            a = A_layer_node_right(a, MAX)
            b = A_layer_node_right(b, MAX)
        elif a < 0 :
            a = A_layer_node_left(a, MIN)
            b = A_layer_node_left(b, MIN)
    elif z > prob_p :
        a = a
        b = b
    return a, b

def A_layer_compromise_function(a, b) :  # A layer  중에서 opposite orientation 에서 일어나는 변동 현상 
    z = random.random()
    if z < (1-prob_p) :
        if a * b == -1 :
            if z < ((1-prob_p)/2) :
                a = 1
                b = 1
            elif z > ((1-prob_p)/2) :
                a = -1
                b = -1
        elif a > b :
            a = A_layer_node_left(a, MIN)
            b = A_layer_node_right(b, MAX)         
        elif a < b :
            a = A_layer_node_right(a, MAX)
            b = A_layer_node_left(b, MIN) 
    elif z > (1-prob_p) :
        a = a
        b = b
    return a, b

def AB_layer_persuasion_function(a, b) :  # A-B layer 중에서 same orientation 에서 일어나는  변동 현상
    z = random.random()
    if z < prob_p :
        if a > 0 :
            a = A_layer_node_right(a, MAX)
            b = b
        elif a < 0 :
            a = A_layer_node_left(a, MIN)
            b = b
    elif z > prob_p :
        a = a
        b = b
    return a, b

def AB_layer_compromise_function(a, b) :  # A-B layer  중에서 opposite orientation 에서 일어나는 변동 현상 
    z = random.random()
    if z < (1-prob_p) :
        if a * b == -1 :
            if z < ((1-prob_p)/2) :
                a = 1
                b = b
            elif z > ((1-prob_p)/2) :
                a = -1
                b = b
        elif a > b :
            a = A_layer_node_left(a, MIN)
            b = b      
        elif a < b :
            a = A_layer_node_right(a, MAX)
            b = b 
    elif z > (1-prob_p) :
        a = a
        b = b
    return a, b

        
def AS_model_function(a) :   # B layer 에서 일어나는 변동 현상
    z = random.random()
    if z < prob_beta :
        a = -a 
    return a

    
def A_layer_dynamics() :    # A_layer 다이내믹스, 감마 적용 및 설득/타협 알고리즘 적용
    for i, j in sorted(A_edges.edges()) :
        if A[i] * A[j] > 0 :
            A[i] = A_layer_persuasion_function(A[i], A[j])[0]
            A[j] = A_layer_persuasion_function(A[i], A[j])[1]
        elif A[i] * A[j] < 0 :
            A[i] = A_layer_compromise_function(A[i], A[j])[0]
            A[j] = A_layer_compromise_function(A[i], A[j])[1]
    for i, j in sorted(AB_edges) :    
        if A[j] * B[i] > 0 :
            A[j] = AB_layer_persuasion_function(A[j], B[i])[0]
            B[i] = AB_layer_persuasion_function(A[j], B[i])[1]
        elif A[j] * B[i] < 0 :
            A[j] = AB_layer_compromise_function(A[j], B[i])[0]
            B[i] = AB_layer_compromise_function(A[j], B[i])[1]
    return A, prob_p, prob_q

def B_layer_dynamics() :  # B_layer 다이내믹스, 베타 적용 및 언어데스 알고리즘 적용
    global prob_beta
    for i in sorted(B_edges.nodes) :
        opposite = []
        intra_edge_number = len(sorted(nx.all_neighbors(B_edges, i)))
        inter_edge_number = len(AB_neighbor[i])
        for j in range(intra_edge_number) :
            if B[i] * B[sorted(nx.all_neighbors(B_edges, i))[j]] < 0 :
                opposite.append(1)
        for j in range(inter_edge_number):
            if B[i] * A[AB_neighbor[i][j]] < 0 :
                opposite.append(1)
        prob_beta = (sum(opposite) / (inter_edge_number + intra_edge_number))**beta
        B[i] = AS_model_function(B[i])        
    return B

def interconnected_dynamics(result) :
    global Probability_plus, passtime, B_plus, A_plus, Flow_prob_beta, Flow_Prob_Beta, Fraction_A_state, Fraction_B_state
    starttime = time.time()                          # 시간 측정
    total = 0
    while True :
        A_layer_dynamics()
        B_layer_dynamics()
        drawing_graph(result)
        total += 1 
        if (np.all(A > 0) == 1 and np.all(B > 0) == 1) or (np.all(A < 0)== 1 and np.all(B < 0)== 1) or (total == limited_time) :
            break
    endtime = time.time()
    passtime = endtime - starttime 
    B_plus = sum(B)        # B_layer 노드의 합
    A_plus = sum(A)        # A_layer 노드의 합
    return A, B, A_plus, B_plus, passtime
 
def static_variable(a, b) :
    global ganma, beta, prob_p, prob_q, beta
    ganma = a
    beta = b
    prob_p = ganma / (1+ganma)           # 확률 p
    prob_q = 1 - prob_p                  # 확률 q
    return ganma, beta, prob_p, prob_q


def making_A_layer_graph() :
    global A_pair, A_nodecolor
    A_pair = []
    A_nodecolor = []
    inter_net.add_layer('A_layer')
    for i in sorted(A_edges.nodes) :
        inter_net.add_node(i)
        if A[i] == 2 :
            A_nodecolor.append((i, 'A_layer', 2))
        elif A[i] == 1 :
            A_nodecolor.append((i, 'A_layer', 1))
    for i, j in sorted(A_edges.edges) :
        inter_net[i, j, 'A_layer'] = 1
        A_pair.append(((i,'A_layer'),(j,'A_layer')))
    return inter_net

# "rule":"edgeweight","colormap":"jet","scaleby":0.1

def making_B_layer_graph() :
    global B_pair, B_nodecolor
    B_pair = []
    B_nodecolor = []
    inter_net.add_layer('B_layer')
    for i in sorted(B_edges.nodes) :
        inter_net.add_node(i)
        if B[i] == 1 :
            B_nodecolor.append((i, 'B_layer', 1))
        elif B[i] == -1 :
            B_nodecolor.append((i, 'B_layer', -1))
    for i, j in sorted(B_edges.edges) :
        inter_net[i, j, 'B_layer'] = 1
        B_pair.append(((i,'B_layer'),(j,'B_layer')))
    return inter_net

def inter_edge_graph() :
    global inter_pair, inter_net
    inter_pair = []
    inter_net = MultilayerNetwork(aspects = 1)
    making_A_layer_graph()
    making_B_layer_graph()
    for i, j in sorted(AB_edges) :
        inter_net[j, 'A_layer'][i, 'B_layer'] = 1
        inter_pair.append(((j,'A_layer'),(i,'B_layer')))
    return inter_net

def edgecolordic() :
    global edgeColorDict
    edgeColorDict={}
    for i in range(int(len(A_edges.edges))) :
        edgeColorDict[A_pair[i]] = 'r'
    for i in range(int(len(B_edges.edges))) :
        edgeColorDict[B_pair[i]] = 'b'
    for i in range(int(len(AB_edges))) :
        edgeColorDict[inter_pair[i]] = 'g'
    return edgeColorDict

## nodeColorDict={(0,0):"r",(1,0):"r",(0,1):"r"}
def nodecolordic() :
    global nodeColorDict
    nodeColorDict={}
    for i in range(int(len(A_nodecolor))) :
        if A_nodecolor[i][2] == 2 :
            nodeColorDict[(A_nodecolor[i][0], 'A_layer')] = 'red'
        elif A_nodecolor[i][2] == 1 :
            nodeColorDict[(A_nodecolor[i][0], 'A_layer')] = 'hotpink'
        elif A_nodecolor[i][2] == -1 :
            nodeColorDict[(A_nodecolor[i][0], 'A_layer')] = 'deepskyblue'
        elif A_nodecolor[i][2] == -2 :
            nodeColorDict[(A_nodecolor[i][0], 'A_layer')] = 'blue'
    for i in range(int(len(B_nodecolor))) :
        if B_nodecolor[i][2] == 1 :
            nodeColorDict[(B_nodecolor[i][0], 'B_layer')] = 'hotpink'
        elif B_nodecolor[i][2] == -1 :
            nodeColorDict[(B_nodecolor[i][0], 'B_layer')] = 'deepskyblue'            
    return nodeColorDict

def nodecoordsdic() :
    global nodeCoordsDict
    nodeCoordsDict = {}
    for i in sorted(A_edges.nodes) :
        nodeCoordsDict[i, 'A_layer'] = (random.random(), random.random())
    for i in sorted(B_edges.nodes) :
        nodeCoordsDict[i, 'B_layer'] = (random.random(), random.random())
    return nodeCoordsDict

def nodelayercoordsdic() :
    global nodelayerCoordsDict
    nodelayerCoordsDict = {'A_layer': (1, 1),'B_layer': (0, 0) }
    return nodelayerCoordsDict

def drawing_graph(result) :    # drawing_graph("inter_net_BA_leaderasdf.png")
    inter_edge_graph()
    fig = draw(inter_net, layout='spring', show=False, layergap=1.3,
            nodelayerCoords=nodelayercoordsdic(),  
           layerPadding=0.05, alignedNodes=True, ax=None, 
           layerColorDict={'A_layer' :'pink', 'B_layer' :'steelblue'}, layerColorRule={}, defaultLayerColor='#29b7c1',
           edgeColorDict= edgecolordic(),
           edgeWidthDict={}, edgeWidthRule={}, defaultEdgeWidth=0.05, 
           edgeStyleDict={}, edgeStyleRule={'inter': ':', 'intra': '-', 'rule': 'edgetype'}, defaultEdgeStyle='-',
           nodeLabelDict= {}, nodeLabelRule={}, defaultNodeLabel=None, 
           nodeColorDict= nodecolordic(), nodeColorRule={}, defaultNodeColor=None,
           nodeLabelColorDict = {} ,nodeLabelColorRule={}, defaultNodeLabelColor='k',
           nodeSizeDict={}, nodeSizeRule={'scalecoeff': 0.3, 'rule': 'scaled'}, defaultNodeSize=None)
    fig.savefig(result)
    image = plt.imread(result)
    return image

def animation_interconnected_dynamics(result) :
    A_layer_dynamics()
    B_layer_dynamics()
    drawing_graph(result)
    return drawing_graph(result)

def plot_movie_mp4(image_array, result):
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])
    def animate(i):
        im.set_array(image_array[i])
        return (im,)
    anim = animation.FuncAnimation(fig, animate, frames=len(image_array), repeat = False, interval=500)
    anim.save(result)
    display(HTML(anim.to_html5_video()))
    

In [128]:
from pymnet import *
import Setting_Simulation_Value
import matplotlib.pyplot as plt
import Layer_A_Modeling
import Layer_B_Modeling


from pymnet import *
import Setting_Simulation_Value
import Layer_A_Modeling
import Layer_B_Modeling


class Interconnected_Layer_Modeling:
    def __init__(self):
        self.SS = Setting_Simulation_Value
        
    def making_layer_A_graph(self, layer_A, interconnected_network):
        interconnected_network.add_layer('layer_A')
        for i in sorted(layer_A.A_edges.nodes):
            interconnected_network.add_node(i)
        for i, j in sorted(layer_A.A_edges.edges):
            interconnected_network[i, j, 'layer_A'] = 1
            
    def making_layer_B_graph(self, layer_B, interconnected_network):
        interconnected_network.add_layer('layer_B')
        for i in sorted(layer_B.B_edges.nodes):
            interconnected_network.add_node(i)
        for i, j in sorted(layer_B.B_edges.edges):
            interconnected_network[i, j, 'layer_B'] = 1
        

    def making_interconnected_layer(self, layer_A, layer_B):
        interconnected_network = MultilayerNetwork(aspects=1)
        self.making_layer_A_graph(layer_A, interconnected_network)
        self.making_layer_B_graph(layer_B, interconnected_network)
        for i, j in sorted(layer_A.AB_edges):
            interconnected_network[j, 'layer_A'][i, 'layer_B'] = 1
        return interconnected_network


if __name__ == "__main__":
    print("Interconnected Layer Modeling")
    layer_A = Layer_A_Modeling.Layer_A_Modeling()
    layer_B = Layer_B_Modeling.Layer_B_Modeling()
    ILM = Interconnected_Layer_Modeling()
    result = ILM.making_interconnected_layer(layer_A, layer_B)
    fig = draw(result, layout='spring', show=False, layergap=1.3, layerPadding=0.05, alignedNodes=True, ax=None, 
               layerColorDict={'layer_A' :'pink', 'layer_B' :'steelblue'}, layerColorRule={}, defaultLayerColor='#29b7c1',
               edgeWidthDict={}, edgeWidthRule={}, defaultEdgeWidth=0.05, 
               edgeStyleDict={}, edgeStyleRule={'inter': ':', 'intra': '-', 'rule': 'edgetype'}, defaultEdgeStyle='-',
               nodeLabelDict= {}, nodeLabelRule={}, defaultNodeLabel=None, 
               nodeLabelColorDict = {} ,nodeLabelColorRule={}, defaultNodeLabelColor='k',
               nodeSizeDict={}, nodeSizeRule={'scalecoeff': 0.3, 'rule': 'scaled'}, defaultNodeSize=None)
    fig.savefig('result')
    print("Operating finished")

Interconnected Layer Modeling
Operating finished


In [117]:
node_info= [{'node_number': i, 'layer': 'A', 'location': (random.random(), random.random()), 'state' : A[i]}
                    for i in sorted(A_edges.nodes)]
node_info = pd.DataFrame(node_info, columns = ['node_number', 'layer', 'location', 'state'])

In [121]:
node_info.state[2]

1

In [45]:
A_layer_config(state = [1, 2], node = 8, edge = 3, Max = 2, Min = -2)
B_layer_config(state = [-1], node = 8, edge = 3, inter_edge= 2)
A_layer_model = select_layer_A_model(1)
B_layer_model = select_layer_B_model(1)
making_interconnected_edges()
drawing_graph('exercise.png')

C:\Users\Purple\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       ...,

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1.

In [50]:
node_dic = {'layer': 'A', 'node_number': 1,
                    'location': (random.random(), random.random())}

In [51]:
node_dic['layer']

'A'

In [52]:
A_pair

[((0, 'A_layer'), (1, 'A_layer')),
 ((0, 'A_layer'), (6, 'A_layer')),
 ((0, 'A_layer'), (7, 'A_layer')),
 ((1, 'A_layer'), (5, 'A_layer')),
 ((1, 'A_layer'), (6, 'A_layer')),
 ((2, 'A_layer'), (3, 'A_layer')),
 ((2, 'A_layer'), (5, 'A_layer')),
 ((2, 'A_layer'), (7, 'A_layer')),
 ((4, 'A_layer'), (3, 'A_layer')),
 ((4, 'A_layer'), (5, 'A_layer')),
 ((4, 'A_layer'), (6, 'A_layer')),
 ((7, 'A_layer'), (3, 'A_layer'))]

In [33]:
nodeCoordsDict

{(0, 'A_layer'): (0.8559895933008618, 0.8246407384883555),
 (1, 'A_layer'): (0.9713053223666661, 0.5259548962490056),
 (2, 'A_layer'): (0.7795258334783499, 0.07446145300639173),
 (3, 'A_layer'): (0.6728891543778063, 0.9833375918959827),
 (4, 'A_layer'): (0.6517285770400555, 0.11010926340971838),
 (5, 'A_layer'): (0.7881948927698983, 0.9852398806216341),
 (6, 'A_layer'): (0.4997180912283363, 0.5113462215543746),
 (7, 'A_layer'): (0.8162489986577985, 0.3323471443545555),
 (8, 'A_layer'): (0.32772545431984035, 0.8988589630071863),
 (9, 'A_layer'): (0.2684687112384947, 0.126940397120906),
 (10, 'A_layer'): (0.8416915429563866, 0.44407857339403656),
 (11, 'A_layer'): (0.38380011457187013, 0.5085241364897266),
 (12, 'A_layer'): (0.29079871942949687, 0.9269406458222089),
 (13, 'A_layer'): (0.9503373024676268, 0.05961117704718377),
 (14, 'A_layer'): (0.1473212456283597, 0.3491582100509547),
 (15, 'A_layer'): (0.37721607974795246, 0.06544264958715584),
 (16, 'A_layer'): (0.9647405356115187, 0.0

In [28]:
inter_pair

[((1, 'A_layer'), (0, 'B_layer')),
 ((2, 'A_layer'), (1, 'B_layer')),
 ((6, 'A_layer'), (2, 'B_layer')),
 ((4, 'A_layer'), (3, 'B_layer')),
 ((5, 'A_layer'), (4, 'B_layer')),
 ((7, 'A_layer'), (5, 'B_layer')),
 ((0, 'A_layer'), (6, 'B_layer')),
 ((3, 'A_layer'), (7, 'B_layer'))]

In [14]:
inter_net

In [9]:
mnet = MultilayerNetwork(aspects=1)
mnet[1,2,'a','b'] = 1
list(mnet)
mplex = MultiplexNetwork(couplings="none")
mplex[1,'a'][2,'a']=1

In [10]:
webplot(mplex, outputfile=None)

'\n<script src="http://d3js.org/d3.v3.min.js"></script>\n<script>\n var mpnet = JSON.parse(\'{"nodes": [{"name": 1}, {"name": 2}], "layers": [{"name": "a"}], "links": [{"source": 0, "target": 1, "value": 1, "layer": 0}]}\');\n\n var color = d3.scale.category20();\n var svg_layer=[];\n var node_layer=[];\n var link_layer=[];\n var layer_label=[];\n\n // Calculate size for the figure\n var width = Math.sqrt(mpnet.nodes.length)*70;//500;\n var height = 4/6*width;\n var fontsize=Math.max(width*0.05,16)\n\n var force = d3.layout.force()\n                      .charge(-120)\n                      .linkDistance(30)\n                      .size([width, height])\n                      .nodes(mpnet.nodes)\n                      .links(mpnet.links)\n                      .start();\n\n var nlayers=mpnet.layers.length;\n\n for (var layer=nlayers-1;layer>=0;layer--){\n  svg_layer[layer] = d3.select("body").append("svg")\n                     .attr("layer",0)\n                     .style("position","

In [11]:
len(A_edges.edges)

12

In [36]:
A_layer_config(state = [1, 2], node = 128, edge = 3, Max = 2, Min = -2)
B_layer_config(state = [-1], node = 128, edge = 3, inter_edge= 1)
A_layer_model = select_layer_A_model(2)
B_layer_model = select_layer_B_model(1)
making_interconnected_edges()
static_variable(0.5, 2)
fig = plt.figure()
ims = [np.array(drawing_graph('dynamic_image.png'))]
limited_time = 1000
total = 0
while True :
    im = animation_interconnected_dynamics('dynamic_image.png')
    ims.append(np.array(im))
    total += 1 
    if (np.all(A > 0) == 1 and np.all(B > 0) == 1) or (np.all(A < 0)== 1 and np.all(B < 0)== 1) or (total == limited_time) :
        break
im = animation_interconnected_dynamics('dynamic_image.png')
ims.append(np.array(im))
IMS = np.array(ims)
plot_movie_mp4(IMS,'dynamic_images_no_leader(128(BA), 128(RR), ganma=0.5, beta=2).mp4')

In [4]:
import networkx as nx
import numpy as np
from enthought.mayavi import mlab

# some graphs to try
#H=nx.krackhardt_kite_graph()
#H=nx.Graph();H.add_edge('a','b');H.add_edge('a','c');H.add_edge('a','d')
#H=nx.grid_2d_graph(4,5)
H=nx.cycle_graph(20)

# reorder nodes from 0,len(G)-1
G=nx.convert_node_labels_to_integers(H)
# 3d spring layout
pos=nx.spring_layout(G,dim=3)
# numpy array of x,y,z positions in sorted node order
xyz=np.array([pos[v] for v in sorted(G)])
# scalar colors
scalars=np.array(G.nodes())+5

mlab.figure(1, bgcolor=(0, 0, 0))
mlab.clf()

pts = mlab.points3d(xyz[:,0], xyz[:,1], xyz[:,2],
                    scalars,
                    scale_factor=0.1,
                    scale_mode='none',
                    colormap='Blues',
                    resolution=20)

pts.mlab_source.dataset.lines = np.array(G.edges())
tube = mlab.pipeline.tube(pts, tube_radius=0.01)
mlab.pipeline.surface(tube, color=(0.8, 0.8, 0.8))

mlab.savefig('mayavi2_spring.png')
# mlab.show() # interactive window

ModuleNotFoundError: No module named 'enthought'

In [2]:
from networkx import *

# plain graph

G=complete_graph(5)   # start with K5 in networkx
A=to_agraph(G)        # convert to a graphviz graph
A.layout()            # neato layout
A.draw("k5.ps")

NameError: name 'to_agraph' is not defined

In [54]:
A_layer_config(state = [1, 2], node = 128, edge = 3, Max = 2, Min = -2)
B_layer_config(state = [-1], node = 128, edge = 3, inter_edge= 1)
A_layer_model = select_layer_A_model(2)
B_layer_model = select_layer_B_model(1)
making_interconnected_edges()
static_variable(0.5, 2)
fig = plt.figure()
ims = [np.array(drawing_graph('dynamic_image.png'))]
limited_time = 1000
total = 0
while True :
    im = animation_interconnected_dynamics('dynamic_image.png')
    ims.append(np.array(im))
    total += 1 
    if (np.all(A > 0) == 1 and np.all(B > 0) == 1) or (np.all(A < 0)== 1 and np.all(B < 0)== 1) or (total == limited_time) :
        break
im = animation_interconnected_dynamics('dynamic_image.png')
ims.append(np.array(im))
IMS = np.array(ims)
plot_movie_mp4(IMS,'dynamic_images_no_leader(128（BA), 128(RR), ganma=0.5, beta=2).mp4')

C:\Users\Purple\Anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
